# Memory experiment replication

This notebook aims at replicating the results from the memory experiment already implemented in `stim`. It serves as an example on how to use the `tqec` Python library as well as a large functional test, testing most of the `tqec` library on its external results rather than on its internal implementation.

## Mandatory imports

In [ ]:
import cirq
import matplotlib.pyplot as plt
import sinter
import stim
from stimcirq import cirq_circuit_to_stim_circuit

from tqec.templates.constructions.qubit import QubitSquareTemplate
from tqec.templates.scale import Dimension, LinearFunction
from tqec.detectors.operation import make_observable, make_shift_coords
from tqec.detectors.transformer import transform_to_stimcirq_compatible
from tqec.enums import PlaquetteOrientation
from tqec.generation.circuit import generate_circuit
from tqec.noise_models import (
    AfterCliffordDepolarizingNoise,
    AfterResetFlipNoise,
    BeforeMeasurementFlipNoise,
)
from tqec.plaquette.plaquette import PlaquetteList
from tqec.plaquette.library import (
    XXPlaquetteList,
    XXXXPlaquetteList,
    ZZPlaquetteList,
    ZZZZPlaquetteList,
)
from tqec.position import Shape2D

## Normalisation and noisyness

Once the quantum error correction circuit implemented, we still need to apply two passes to obtain a circuit ready to be translated by the `stimcirq`.

The first pass normalises the `cirq.Circuit` produced. This pass was performing several modifications before, but is now simply removing potential empty `cirq.Moment` instances from the `cirq.Circuit` instance.

The second pass applies the noise model(s) we want to consider in the `stim` simulation.

In [ ]:
def normalise_circuit(circuit: cirq.Circuit) -> cirq.Circuit:
    ordered_transformers = [
        cirq.drop_empty_moments,
    ]
    for transformer in ordered_transformers:
        circuit = transformer(circuit)
    return circuit


def to_noisy_circuit(circuit: cirq.Circuit, noise_level: float) -> cirq.Circuit:
    noise_models = [
        AfterCliffordDepolarizingNoise(noise_level),
        AfterResetFlipNoise(noise_level),
        BeforeMeasurementFlipNoise(noise_level),
    ]
    for nm in noise_models:
        circuit = circuit.with_noise(nm)
    return circuit

## Building the Circuit instance

The main goal of the `tqec` library is to construct the `cirq.Circuit` instance representing the QEC experiment we are interest about. In this notebook, we want to perform a simple logical qubit memory experiment.

The different pre-defined Plaquette instances `XXPlaquetteList`, ... implement by default the memory experiment, so we can use them without worrying about the quantum circuits actually executed.

In [ ]:
def generate_cirq_circuit_tqec(k: int, repetitions: int) -> cirq.Circuit:
    # 1. Define the template.
    # Here, the template is directly constructed to the requested dimensions, so we do not need to
    # re-scale it using the Template.to_scale method.
    template = QubitSquareTemplate(Dimension(k, LinearFunction(2)))

    # 2. Define the different plaquettes that will be used on the template defined above.
    # As noted in the text description above this function, the plaquette instances in the list below
    # implement by default the memory experiment we want to perform. That is the reason why there is no
    # tedious quantum circuit definition here.
    # The following time schedules are used accross plaquettes:
    # - Initial reset index is 1
    # - (H gate for X-stabilizers is 2)
    # - CNOTs indices are 3, 4, 5, 6
    # - (H gate for X-stabilizers is 7)
    # - Measurement index is 8
    plaquettes: list[PlaquetteList] = [
        XXPlaquetteList(
            PlaquetteOrientation.UP, [1, 2, 5, 6, 7, 8], include_detector=False
        ),
        ZZPlaquetteList(PlaquetteOrientation.LEFT, [1, 5, 6, 8]),
        XXXXPlaquetteList([1, 2, 3, 4, 5, 6, 7, 8], include_detector=False),
        ZZZZPlaquetteList([1, 3, 4, 5, 6, 8]),
        ZZPlaquetteList(PlaquetteOrientation.RIGHT, [1, 3, 4, 8]),
        XXPlaquetteList(
            PlaquetteOrientation.DOWN, [1, 2, 3, 4, 7, 8], include_detector=False
        ),
    ]

    # 3. Define the layer modifiers.
    # In the memory experiment, we want to repeat the second layer `repetitions` times in order to
    # let the experiment run for an extended period of time.
    # This is done by encapsulating the circuit generated for the second layer into a
    # cirq.CircuitOperation that natively supports repeting an operation efficiently, and that is
    # natively recognized by the `tqec` library.
    def make_repeated_layer(circuit: cirq.Circuit) -> cirq.Circuit:
        # Note: we do not care on which qubit it is applied, but we want a SHIFT_COORDS instruction
        #       to be inserted somewhere in the repetition loop. It is inserted at the beginning.
        any_qubit = next(iter(circuit.all_qubits()), None)
        assert (
            any_qubit is not None
        ), "Could not find any qubit in the given Circuit instance."
        circuit_to_repeat = cirq.Circuit([make_shift_coords(0, 0, 1)]) + circuit
        repeated_circuit_operation = cirq.CircuitOperation(
            circuit_to_repeat.freeze()
        ).repeat(repetitions)
        return cirq.Circuit([repeated_circuit_operation])

    layer_modificators = {1: make_repeated_layer}

    # 4. Actually create the cirq.Circuit instance by concatenating the circuits generated
    # for each layer and potentially modified by the modifiers defined above.
    circuit = cirq.Circuit()
    for layer_index in range(3):
        layer_circuit = generate_circuit(
            template,
            [plaquette_list.plaquettes[layer_index] for plaquette_list in plaquettes],
        )
        layer_circuit = normalise_circuit(layer_circuit)
        circuit += layer_modificators.get(layer_index, lambda circ: circ)(layer_circuit)

    # 5. Define the observable.
    # The observable is defined and added to the cirq.Circuit instance just here.
    # We assume that each plaquette has the same shape (i.e., needs the same number of qubits on the X and
    # Y dimensions). XX and ZZ stabilizers have been artificially made 3x3 plaquettes for this purpose. This
    # assumption will eventually need to be lifted.
    plaquette_shape: Shape2D = plaquettes[0].plaquettes[0].shape
    assert all(
        p.shape == plaquette_shape
        for plaquette_list in plaquettes
        for p in plaquette_list.plaquettes
    ), "All plaquettes should have exactly the same shape for the moment."
    origin = cirq.GridQubit(plaquette_shape.y - 1, plaquette_shape.x - 1)
    circuit.append(
        cirq.Moment(
            make_observable(
                origin,
                [
                    (cirq.GridQubit(0, i * (plaquette_shape.x - 1)), -1)
                    for i in range(2 * k + 1)
                ],
            )
        )
    )

    # 6. Due to limitations in the API provided by Stim, the DetectorGate and ObservableGate instances
    # in the circuit only contain local measurement records, an internal representation that is not
    # understood by Stim. The fill_in_global_record_indices function replaces these local measurement
    # records by global ones that are understood by Stim.
    # return circuit
    return transform_to_stimcirq_compatible(circuit)

## Normalise the API between `stim` and `tqec`

Both libraries have slight variations in their API that makes their side-by-side use error-prone. To avoid typos and inattention issues, we create 2 functions with the same API that take care of handling the small discrepencies for us.

In [ ]:
def generate_stim_circuit_tqec(
    code_distance: int, noise_level: float, repetitions: int
) -> stim.Circuit:
    circuit = generate_cirq_circuit_tqec((code_distance - 1) // 2, repetitions)
    if abs(noise_level) > 1e-12:
        circuit = to_noisy_circuit(circuit, noise_level)
    else:
        print(
            f"Warning: a noise level below 1e-12 ({noise_level}) has been detected. Noise is disabled. If this was intentional, please remove this code."
        )

    return cirq_circuit_to_stim_circuit(circuit)

In [ ]:
def generate_stim_circuit_stim(
    code_distance: int, noise_level: float, repetitions: int
) -> stim.Circuit:
    return stim.Circuit.generated(
        "surface_code:rotated_memory_z",
        rounds=repetitions + 1,
        distance=code_distance,
        after_clifford_depolarization=noise_level,
        after_reset_flip_probability=noise_level,
        before_measure_flip_probability=noise_level,
        before_round_data_depolarization=0,
    )

## TQEC plots

We should finally be ready to perform the `stim` simulations and plot the results to check if our `tqec`-generated QEC memory experiment obtains the same results as the `stim`-generated one.

In [ ]:
surface_code_tasks_tqec = [
    sinter.Task(
        circuit=generate_stim_circuit_tqec(d, noise, 3 * d),
        json_metadata={"d": d, "r": 3 * d, "p": noise},
    )
    for d in [3, 5, 7, 9, 11]
    for noise in [0.001, 0.002, 0.005, 0.009, 0.01, 0.011, 0.012, 0.013, 0.014]
]

collected_surface_code_stats_tqec: list[sinter.TaskStats] = sinter.collect(
    num_workers=20,
    tasks=surface_code_tasks_tqec,
    decoders=["pymatching"],
    max_shots=1_000_000,
    max_errors=5_000,
    print_progress=False,
)

In [ ]:
fig, ax = plt.subplots(1, 1)
sinter.plot_error_rate(
    ax=ax,
    stats=collected_surface_code_stats_tqec,
    x_func=lambda stat: stat.json_metadata["p"],
    group_func=lambda stat: stat.json_metadata["d"],
    failure_units_per_shot_func=lambda stat: stat.json_metadata["r"],
)
ax.loglog()
ax.set_title("TQEC: Surface Code Error Rates per Round under Circuit Noise")
ax.set_xlabel("Phyical Error Rate")
ax.set_ylabel("Logical Error Rate per Round")
ax.grid(which="major")
ax.grid(which="minor")
ax.legend()
fig.set_dpi(120)  # Show it bigger

## Stim plots

In [ ]:
surface_code_tasks_stim = [
    sinter.Task(
        circuit=generate_stim_circuit_stim(d, noise, 3 * d),
        json_metadata={"d": d, "r": 3 * d, "p": noise},
    )
    for d in [3, 5, 7, 9, 11]
    for noise in [0.001, 0.002, 0.005, 0.009, 0.01, 0.011, 0.012, 0.013, 0.014]
]

collected_surface_code_stats_stim: list[sinter.TaskStats] = sinter.collect(
    num_workers=20,
    tasks=surface_code_tasks_stim,
    decoders=["pymatching"],
    max_shots=1_000_000,
    max_errors=5_000,
    print_progress=False,
)

In [ ]:
fig, ax = plt.subplots(1, 1)
sinter.plot_error_rate(
    ax=ax,
    stats=collected_surface_code_stats_stim,
    x_func=lambda stat: stat.json_metadata["p"],
    group_func=lambda stat: stat.json_metadata["d"],
    failure_units_per_shot_func=lambda stat: stat.json_metadata["r"],
)
ax.loglog()
ax.set_title("Stim: Surface Code Error Rates per Round under Circuit Noise")
ax.set_xlabel("Phyical Error Rate")
ax.set_ylabel("Logical Error Rate per Round")
ax.grid(which="major")
ax.grid(which="minor")
ax.legend()
fig.set_dpi(120)  # Show it bigger

## Circuit visualisation

Some debugging relicates that are left here if you want to visualise/compare the quantum circuits generated.

In [ ]:
export_stim_files = True

stim_circuit_stim = generate_stim_circuit_stim(3, 0.001, 2)
stim_circuit_tqec = generate_stim_circuit_tqec(3, 0.001, 2)
if export_stim_files:
    with open("./stim.stim", "w") as f:
        f.write(str(stim_circuit_stim))
    with open("./tqec.stim", "w") as f:
        f.write(str(stim_circuit_tqec))
    print("Export done!")

In [ ]:
if export_stim_files:
    with open("./tqec.stim", "r") as f:
        stim_circuit_tqec = stim.Circuit(f.read())

In [ ]:
stim_circuit_tqec.diagram("timeslice-svg")

In [ ]:
stim_circuit_stim.diagram("timeslice-svg")